In [ ]:
%%time
import datetime

start = datetime.datetime.now()
print(start)
import os as os

os.chdir("C:/Users/prana/Desktop/directory/WildAlertModels/")
# os.chdir('/home/falco/Desktop/directory/Syndrome_classifier_trainer')
data_path = "C:/Users/prana/Desktop/directory/WildAlertModels/data/"
# data_path = '/home/falco/Desktop/directory/Syndrome_classifier_trainer/data'
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline
import datetime
import os
import pickle
from math import ceil

import app
import config
import nltk
import numpy as np
import pandas as pd
import preprocess
import simplejson as json
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import (GRU, LSTM, BatchNormalization, Bidirectional, Conv1D, Dense, Dropout, Embedding,
                                     GlobalAveragePooling1D, GlobalMaxPooling1D, Input, LeakyReLU, MaxPooling1D,
                                     SpatialDropout1D, concatenate)
from tensorflow.keras.models import Model, Sequential
from textblob import TextBlob
from tqdm import tqdm

# from WRMDpy import Syndrome as Syndrome


print("tensorflow version ", tf.__version__)

import keras

print("keras version ", keras.__version__)
import keras.callbacks
from imblearn.ensemble import (BalancedBaggingClassifier, BalancedRandomForestClassifier, EasyEnsembleClassifier,
                               RUSBoostClassifier)
from imblearn.over_sampling import SMOTE
# Ensembled sampling
from imblearn.under_sampling import NearMiss
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences

plt.style.use("fivethirtyeight")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "serif"

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [ ]:
import requests

path1 = "https://www.wrmd.org/api/v2/ml/training-dataset/clinical-classifications"
# path1 = 'API/link/TrainingData'
raw_data = json.loads(requests.get(path1).text)
raw_data = pd.DataFrame(raw_data)
cond = raw_data.terms.apply(pd.Series)
cond.columns = ["c1", "c2", "c3", "c4", "c5"]
raw_data = pd.concat([raw_data, cond], axis=1)
raw_data["condition_predict"] = raw_data.c1
raw_data.head()

In [ ]:
raw_data.shape

In [ ]:
raw_data.text.iloc[2]

In [ ]:
data = raw_data[["patient_id", "text", "terms"]].explode(column="terms")
data["condition_predict"] = data.terms
data.shape

In [ ]:
len(data.condition_predict.unique())

In [ ]:
data.condition_predict.value_counts()

In [ ]:
data["condition_predict"].value_counts().sort_values(ascending=False).plot(
    kind="barh", title="Number of cases in each clinical classification"
)

In [ ]:
import re

df = data
REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|@,;]")
BAD_SYMBOLS_RE = re.compile("[^0-9a-z #+_]")
STOPWORDS = set(stopwords.words("english"))


def clean_text(text):
    """
    text: a string

    return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(
        " ", text
    )  # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub(
        "", text
    )  # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = text.replace("x", "")
    #    text = re.sub(r'\W+', '', text)
    text = " ".join(
        word for word in text.split() if word not in STOPWORDS
    )  # remove stopwors from text
    return text


df["text"] = df["text"].apply(clean_text)
df["text"] = df["text"].str.replace("\d+", "")

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 70000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 1000
# This is fixed.
EMBEDDING_DIM = 200
tokenizer = Tokenizer(
    num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True
)
tokenizer.fit_on_texts(df["text"].values)
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

In [ ]:
Tokenizer

In [ ]:
pd.DataFrame.from_dict([word_index]).T

In [ ]:
X = tokenizer.texts_to_sequences(df["text"].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print("Shape of data tensor:", X.shape)

In [ ]:
Y = pd.get_dummies(df["condition_predict"]).values
print("Shape of label tensor:", Y.shape)

In [ ]:
def get_training_testing_data(dataframe):
    X_train_df, X_test_df, Y_train_df, Y_test_df = train_test_split(
        dataframe["text"], Y, test_size=0.10, random_state=42
    )
    print(X_train_df.shape, Y_train_df.shape)
    print(X_test_df.shape, Y_test_df.shape)
    return X_train_df, X_test_df, Y_train_df, Y_test_df

In [ ]:
X_train_df, X_test_df, Y_train_df, Y_test_df = get_training_testing_data(dataframe=df)

In [ ]:
X_train_df_t = tokenizer.texts_to_sequences(X_train_df.values)
X_train_df_t = pad_sequences(X_train_df_t, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
X_train_df_t.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.10, random_state=42
)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
X_train.shape

In [ ]:
%%time
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0))
model.add(Dense(11, activation="softmax"))
## softmax is desinged for only one prediction category. define a sigmoid and then use threshold
##
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],
)


epochs = 200
batch_size = 64

history = model.fit(
    X_train_df_t,
    Y_train_df,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5, min_delta=0.0001)],
)

In [ ]:
%%time
model2 = Sequential()
model2.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0))
model2.add(Dense(11, activation="softmax"))
model2.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

epochs = 200
batch_size = 64

history2 = model2.fit(
    X_train_df_t,
    Y_train_df,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5, min_delta=0.0001)],
)

In [ ]:
%%time
model3 = Sequential()
model3.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model3.add(SpatialDropout1D(0.2))
model3.add(LSTM(100, dropout=0.2, recurrent_dropout=0))
model3.add(Dense(11, activation="softmax"))
model3.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)


epochs = 200
batch_size = 64

history3 = model3.fit(
    X_train_df_t,
    Y_train_df,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5, min_delta=0.0001)],
)

### Predictions

In [ ]:
new_case = [data.text.iloc[3414]]
print(new_case)
seq = tokenizer.texts_to_sequences(new_case)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = pd.get_dummies(df["condition_predict"]).columns.tolist()
print("")
print("First Prediction")
# print(pred, labels[np.argmax(pred)])
print(labels[np.argmax(pred)])
print("")
print("Second Prediction")
sorted_pred = sorted(pred[0])
print(labels[list(pred[0]).index(sorted_pred[-2])])

In [ ]:
X_test_df.iloc[0]

In [ ]:
new_case = [X_test_df.iloc[1]]
print(new_case)
seq = tokenizer.texts_to_sequences(new_case)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = pd.get_dummies(df["condition_predict"]).columns.tolist()
print(pred)
print("")
print("First Prediction")
# print(pred, labels[np.argmax(pred)])
print(labels[np.argmax(pred)])
print("")
print("Second Prediction")
sorted_pred = sorted(pred[0])
print(labels[list(pred[0]).index(sorted_pred[-2])])

In [ ]:
pred = model.predict(padded)

In [ ]:
# ?model.predict

In [ ]:
print(labels[np.argmax(pred)])

In [ ]:
sorted_pred[-2]

In [ ]:
labels = pd.get_dummies(df["condition_predict"]).columns.tolist()


def predict_conditions(c):
    new_case = [c]
    # print(new_case)
    seq = tokenizer.texts_to_sequences(new_case)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded, verbose=0)
    # print(pred)
    # print("")
    # print("First Prediction")
    # print(pred, labels[np.argmax(pred)])
    # print(labels[np.argmax(pred)])
    # print("")
    # print("Second Prediction")
    sorted_pred = sorted(pred[0])
    # print(labels[list(pred[0]).index(sorted_pred[-2])])
    return pd.Series(
        [labels[np.argmax(pred)], labels[list(pred[0]).index(sorted_pred[-2])]]
    )

In [ ]:
print(labels)

In [ ]:
# from tqdm import tqdm
# tqdm.pandas()
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas(desc="PROGRESS>>>")

In [ ]:
%%time
pp = X_test_df.progress_apply(predict_conditions)

In [ ]:
pp.columns = ["First", "Second"]

In [ ]:
df.head()

In [ ]:
pp["True_condition"] = df.iloc[X_test_df.index.values.tolist()][
    "condition_predict"
].values
pp.head()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

confm = confusion_matrix(y_pred=pp.First, y_true=pp.True_condition)
df_cm = pd.DataFrame(confm, index=labels, columns=labels)
ax = sns.heatmap(df_cm, cmap="Oranges", annot=True)
plt.show()

In [ ]:
X_test.shape

In [ ]:
plt.subplots(figsize=(15, 15))
confm = confusion_matrix(y_pred=pp.First, y_true=pp.True_condition, normalize="true")
df_cm = pd.DataFrame(confm, index=labels, columns=labels)
ax = sns.heatmap(df_cm, mask=confm == 0, cmap="Oranges", annot=True)
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title("Performance on test data (n = 1748)")
plt.tight_layout()
plt.savefig(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/outputs/LSTM_02_v2_03_30_2023.png",
    dpi=600,
)
plt.show()

In [ ]:
labels = pd.get_dummies(df["condition_predict"]).columns.tolist()


def predict_conditions(c):
    new_case = [c["text"]]
    # print(new_case)
    seq = tokenizer.texts_to_sequences(new_case)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded, verbose=0)
    # print(pred)
    # print("")
    # print("First Prediction")
    # print(pred, labels[np.argmax(pred)])
    # print(labels[np.argmax(pred)])
    # print("")
    # print("Second Prediction")
    sorted_pred = sorted(pred[0])
    # print(labels[list(pred[0]).index(sorted_pred[-2])])
    return pd.Series(
        [
            c["patient_id"],
            labels[np.argmax(pred)],
            labels[list(pred[0]).index(sorted_pred[-2])],
        ]
    )

In [ ]:
%%time
pp_all = raw_data.progress_apply(predict_conditions, axis=1)
pp_all.columns = ["patient_id", "First", "Second"]

In [ ]:
pp_all.head()

In [ ]:
pp_all["True_condition"] = raw_data.condition_predict
pp_all.head()

In [ ]:
f"{pp_all.shape[0]:,}"

In [ ]:
plt.subplots(figsize=(15, 15))
confm = confusion_matrix(
    y_pred=pp_all.First, y_true=pp_all.True_condition, normalize="true"
)
df_cm = pd.DataFrame(confm, index=labels, columns=labels)
ax = sns.heatmap(df_cm, mask=confm == 0, cmap="Oranges", annot=True)
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title(
    "Performance on training data\nfor first prediction (n ="
    + f"{pp_all.shape[0]:,}"
    + ")"
)
plt.tight_layout()
plt.savefig(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/outputs/LSTM_02_all_v2_03_30_2023.png",
    dpi=600,
)
plt.show()

In [ ]:
def predict_conditions(c):
    new_case = [c["text"]]
    # print(new_case)
    seq = tokenizer.texts_to_sequences(new_case)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded, verbose=0)
    # print(type(list(pred[0])))
    # print("")
    # print("First Prediction")
    # print(pred, labels[np.argmax(pred)])
    # print(labels[np.argmax(pred)])
    # print("")
    # print("Second Prediction")
    sorted_pred = sorted(pred[0])
    # print(labels[list(pred[0]).index(sorted_pred[-2])])
    qq = [
        c["patient_id"],
        labels[np.argmax(pred)],
        np.max(pred),
        labels[list(pred[0]).index(sorted_pred[-2])],
        sorted_pred[-2],
    ] + (list(pred[0]))
    return pd.Series(qq)
    # return pd.Series([c['patient_id'], labels[np.argmax(pred)], np.max(pred), labels[list(pred[0]).index(sorted_pred[-2])], sorted_pred[-2]])

In [ ]:
%%time
pp2_all = raw_data.progress_apply(predict_conditions, axis=1)
pp2_all.columns = ["patient_id", "First", "First_pr", "Second", "Second_pr"] + list(
    range(11)
)

In [ ]:
pp2_all

In [ ]:
raw_data.head()

In [ ]:
pp2_all.head()

In [ ]:
plt.rcParams["figure.figsize"] = (6, 4)
plt.hist(pp2_all.First_pr)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (6, 4)
plt.hist(pp2_all.Second_pr)
plt.show()

In [ ]:
pp2_all["pr_diff"] = pp2_all.First_pr - pp2_all.Second_pr

In [ ]:
plt.hist(pp2_all.pr_diff)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
pp2_all["First_pr"].hist(by=pp2_all["First"])
plt.xlim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
pp2_all["Second_pr"].hist(by=pp2_all["Second"])
plt.xlim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
pp2_all.shape

In [ ]:
raw_data.shape

In [ ]:
raw_data.head()

In [ ]:
pp2_all.head()

In [ ]:
result_df = pd.merge(pp2_all, raw_data, on="patient_id", how="left")
result_df.head()

In [ ]:
result_df.shape

In [ ]:
result_df["number_of_classes"] = result_df["terms"].str.len()
result_df["multiple_classes"] = result_df.number_of_classes.apply(
    lambda x: 1 if x > 1 else 0
)

In [ ]:
result_df["pr_diff_inv"] = 1 - result_df["pr_diff"]

In [ ]:
result_df[["multiple_classes", "pr_diff", "pr_diff_inv"]].head(20)

In [ ]:
result_df.head()

In [ ]:
from sklearn import metrics

plt.rcParams["figure.figsize"] = (8, 6)
fpr, tpr, threshold = metrics.roc_curve(
    result_df.multiple_classes, 1 - result_df.pr_diff
)
roc_auc = metrics.auc(fpr, tpr)
# method I: plt
import matplotlib.pyplot as plt

plt.title("Receiver Operating Characteristic")
plt.plot(fpr, tpr, "b", label="AUC = %0.2f" % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.tight_layout()
plt.savefig(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/outputs/ROC_two_predictions_v2_03_30_2023.png",
    dpi=600,
)
plt.show()

In [ ]:
i = np.arange(len(tpr))
roc = pd.DataFrame(
    {
        "tf": pd.Series(tpr - (1 - fpr), index=i),
        "threshold": pd.Series(threshold, index=i),
    }
)
roc_t = roc.iloc[(roc.tf - 0).abs().argsort()[:1]]

roc_t["threshold"]

In [ ]:
roc_t["threshold"].values[0]

In [ ]:
model.save(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/model_v4_03_30_2023.h5"
)

In [ ]:
%%time
import pickle

with open(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/keras_tokenizer_v4_03_30_2023.pickle",
    "wb",
) as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
labels

In [ ]:
new_case = ["injured, flea ridden, flea, dirt, pale"]
print(new_case)
seq = tokenizer.texts_to_sequences(new_case)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = pd.get_dummies(df["condition_predict"]).columns.tolist()
sorted_pred = sorted(pred[0])
if sorted_pred[-1] - sorted_pred[-2] >= roc_t["threshold"].values[0]:
    print("Final Clinical Classifiaction Prediction")
    print(labels[np.argmax(pred)])
else:
    print("First Prediction")
    print(labels[np.argmax(pred)])
    print("")
    print("Second Prediction")
    print(labels[list(pred[0]).index(sorted_pred[-2])])

In [ ]:
result_df.head()

In [ ]:
result_df2 = result_df

In [ ]:
result_df["multiple_predictions"] = np.where(
    1 - (result_df["First_pr"] - result_df["Second_pr"])
    >= roc_t["threshold"].values[0],
    "Yes",
    "No",
)
result_df["predictions"] = np.where(
    (
        1 - (result_df["First_pr"] - result_df["Second_pr"])
        >= roc_t["threshold"].values[0]
    ),
    result_df["First"].astype(str) + ", " + result_df["Second"],
    result_df["First"],
)

In [ ]:
result_df.multiple_predictions.value_counts()

In [ ]:
condition1 = (result_df.multiple_predictions == "Yes") & (
    result_df.First.isin(["Clinically healthy", "Nonspecific"])
)
condition2 = (result_df.multiple_predictions == "Yes") & (
    result_df.Second.isin(["Clinically healthy", "Nonspecific"])
)

In [ ]:
result_df[condition1].shape

In [ ]:
result_df[condition2].shape

In [ ]:
result_df["multiple_predictions"] = np.where(
    condition1, "No", result_df.multiple_predictions
)
result_df["predictions"] = np.where(
    condition1, result_df["First"], result_df["predictions"]
)

In [ ]:
result_df[
    (result_df.multiple_predictions == "Yes")
    & result_df.First.isin(["Clinically healthy", "Nonspecific"])
].shape

In [ ]:
result_df.multiple_predictions.value_counts()

In [ ]:
result_df["multiple_predictions"] = np.where(
    condition2, "No", result_df.multiple_predictions
)
result_df["predictions"] = np.where(
    condition2, result_df["First"], result_df["predictions"]
)

In [ ]:
result_df.multiple_predictions.value_counts()

In [ ]:
3043 - (154 + 185)

In [ ]:
condition3 = (result_df.multiple_predictions == "Yes") & (
    result_df.predictions.str.contains("Clinically healthy")
)
pattern = "|".join(["Clinically healthy", ","])
result_df["predictions"] = np.where(
    condition3, result_df.predictions.str.replace(pattern, ""), result_df.predictions
)
result_df["multiple_predictions"] = np.where(
    condition3, "No", result_df.multiple_predictions
)

In [ ]:
result_df.to_excel(
    "C:/Users/falco/Desktop/directory/Syndrome_classifier_trainer/outputs/Prediction_comparison_v4_03_30_2023.xlsx"
)

In [ ]:
result_df.head()

In [ ]:
result_df.predictions.str.split(",")

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score, top_k_accuracy_score

In [ ]:
# top_k_accuracy_score(result_df.condition_predict, result_df.predictions.str.split(','), k=2)

In [ ]:
result_df.columns.tolist()

In [ ]:
result_df

In [ ]:
result_df["terms"].apply(pd.Series)

In [ ]:
Y

In [ ]:
raw_data.shape

In [ ]:
pd.get_dummies(df["condition_predict"]).values

In [ ]:
ff = raw_data[["patient_id", "terms"]].explode(column="terms")
ff = pd.concat([ff, pd.get_dummies(ff["terms"])], axis=1)
ff.drop(["terms"], axis=1, inplace=True)
ff = ff.groupby("patient_id").sum().reset_index()
ff

In [ ]:
ff.set_index("patient_id").values

In [ ]:
result_df.drop(["c1", "c2", "c3", "c4", "c5"], axis=1, inplace=True)

In [ ]:
result_df[result_df.patient_id == 2651830]

In [ ]:
result_df[result_df.patient_id == 2651830].values

In [ ]:
result_df[result_df.patient_id == 2651830][[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

In [ ]:
ff[ff.patient_id == 2651830]

In [ ]:
# top_k_accuracy_score(y_true, y_score, k=2)

In [ ]:
y_true_multi = ff.set_index("patient_id").values
y_score = result_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]].values
label_ranking_average_precision_score(y_true_multi, y_score)

In [ ]:
label_ranking_average_precision_score

In [ ]:
temp_result_df = result_df[
    result_df.multiple_predictions == "Yes"
]  # [[0, 1, 2,3,4,5,6,7,8,9,10]].values

In [ ]:
temp_ff = ff[
    ff.patient_id.isin(
        result_df[result_df.multiple_predictions == "Yes"].patient_id.unique().tolist()
    )
]
temp_ff

In [ ]:
t = temp_ff.columns.tolist()
del t[0]

In [ ]:
t

In [ ]:
t_ff_df = pd.merge(temp_ff, temp_result_df, on="patient_id")

In [ ]:
y_true_multi = t_ff_df[t].values
y_score = t_ff_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]].values
label_ranking_average_precision_score(y_true_multi, y_score)

In [ ]:
X_test_df

## Testing ranking_average_precision_score for Test data only

In [ ]:
filter_df_test_data = raw_data.filter(items=X_test_df.index.tolist(), axis=0)

In [ ]:
test_patients = filter_df_test_data.patient_id.unique().tolist()
multiple_clinical_patients = ff.patient_id.isin(
    result_df[result_df.multiple_predictions == "Yes"].patient_id.unique().tolist()
)

In [ ]:
test_results_df = result_df[result_df.patient_id.isin(test_patients)]
test_results_df_multi = test_results_df[test_results_df.multiple_predictions == "Yes"]
test_results_df_multi_patients = test_results_df_multi.patient_id.unique().tolist()

In [ ]:
test_ff = ff[ff.patient_id.isin(test_results_df_multi_patients)]

In [ ]:
test_ff

In [ ]:
t_ff_df_test = pd.merge(test_ff, result_df, on="patient_id", how="left")
t_ff_df_test.shape

In [ ]:
y_true_multi_test = t_ff_df_test[t].values
y_score_test = t_ff_df_test[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]].values
label_ranking_average_precision_score(y_true_multi_test, y_score_test)